In [1]:
import os 
import shutil 
from pathlib import Path
import pandas as pd

**EXTRACTED DATA FROM SEEDED FILE**

In [2]:
# Define paths
downloads_path = str(Path.home() / "Documents" / "major_project")  # Path to the source directory
notebook_dir = os.getcwd()  # Current working directory
raw_table = os.path.join(notebook_dir, "raw_table")  # Destination directory

# Ensure the destination directory exists
os.makedirs(raw_table, exist_ok=True)

# List files in the source directory
files = os.listdir(downloads_path)

# Copy files starting with 'RAW' and are CSV
for file in files:
    if file.startswith("RAW") and file.endswith(".csv"):  # Ensure it's a CSV file
        shutil.copy(os.path.join(downloads_path, file), raw_table)

print(f"CSV files have been copied to {raw_table}")

CSV files have been copied to c:\Users\SabareeshwaranM\Desktop\FINAL_PROJECT\DE_MAJOR_PROJECT-main\raw_table


**RAW TABLES**

In [3]:
# File paths for the CSV files
feedbacks_path = './raw_table/RAW.feedbacks.csv'
learningmaterial_path = './raw_table/RAW.learningmaterials.csv'
quizzes_path = './raw_table/RAW.quizzes.csv'
scores_path = './raw_table/RAW.scores.csv'
users_path = './raw_table/RAW.users.csv'

# Reading CSV files into DataFrames
feedbacks_df = pd.read_csv(feedbacks_path)
learningmaterial_df = pd.read_csv(learningmaterial_path)
quizzes_df = pd.read_csv(quizzes_path)
scores_df = pd.read_csv(scores_path)
users_df = pd.read_csv(users_path)

**CHECKING AND CLEANING NULL VALUES**

In [4]:
#Checking for null values in each DataFrame
print("Null values in feedbacks.csv:")
print(feedbacks_df.isnull().sum())
print("\nNull values in learningmaterial.csv:")
print(learningmaterial_df.isnull().sum())
print("\nNull values in quizzes.csv:")
print(quizzes_df.isnull().sum())
print("\nNull values in scores.csv:")
print(scores_df.isnull().sum())
print("\nNull values in users.csv:")
print(users_df.isnull().sum())


Null values in feedbacks.csv:
_id                      0
employeeId               0
learningMaterialTitle    0
description              0
rating                   0
createdAt                0
__v                      0
dtype: int64

Null values in learningmaterial.csv:
_id            0
title          0
description    0
createdBy      0
createdAt      0
__v            0
dtype: int64

Null values in quizzes.csv:
_id                                   0
title                                 0
questions[0].questionText             0
questions[0].options[0].optionText    0
questions[0].options[1].optionText    0
questions[0].options[2].optionText    0
questions[0].options[0].isCorrect     0
questions[0].options[1].isCorrect     0
questions[0].options[2].isCorrect     0
questions[0].options[0]._id           0
questions[0].options[1]._id           0
questions[0].options[2]._id           0
questions[0]._id                      0
learningMaterial                      0
createdAt                 

**TRANSFORMING THE TABLE AND SAVING AS PREP**

In [5]:
# users_df['email'] = users_df['email'].str.lower()   # Normalize email to lowercase


feedbacks_df['createdAt'] = pd.to_datetime(feedbacks_df['createdAt'])  # Convert to datetime


learningmaterial_df['createdAt'] = pd.to_datetime(learningmaterial_df['createdAt'])


quizzes_df['createdAt'] = pd.to_datetime(quizzes_df['createdAt'])

**Droping Unwanted Coloumns**

In [6]:
# feedbacks_df.dtypes

feedbacks_df.drop(columns=['description', '__v'], inplace=True)

In [7]:

users_df.drop(columns=['__v'], inplace=True)

learningmaterial_df.drop(columns=['description', '__v'], inplace=True)

selected_columns = ['_id', 'title', 'learningMaterial','createdAt']
quizzes_df = quizzes_df.loc[:, selected_columns]

scores_df.drop(columns=['__v'], inplace=True)


In [8]:
output_folder = './prep_table'
os.makedirs(output_folder, exist_ok=True)

# File paths for the new CSVs
users_output_path = os.path.join(output_folder, 'PREP.users.csv')
learningmaterial_output_path = os.path.join(output_folder, 'PREP.learningmaterials.csv')
quizzes_output_path = os.path.join(output_folder, 'PREP.quizzes.csv')
scores_output_path = os.path.join(output_folder, 'PREP.scores.csv')
feedbacks_output_path = os.path.join(output_folder, 'PREP.feedbacks.csv')

# Save the DataFrames to CSV files in the new folder
users_df.to_csv(users_output_path, index=False)
learningmaterial_df.to_csv(learningmaterial_output_path, index=False)
quizzes_df.to_csv(quizzes_output_path, index=False)
scores_df.to_csv(scores_output_path, index=False)
feedbacks_df.to_csv(feedbacks_output_path, index=False)

print(f"DataFrames successfully saved in '{output_folder}'")


DataFrames successfully saved in './prep_table'


**FACT AND DIMENSIONAL TABLES**

In [9]:
# Load the data
df_selected_scores = pd.read_csv("./prep_table/PREP.scores.csv")
df_selected_quizzes = pd.read_csv("./prep_table/PREP.quizzes.csv")
df_feedbacks = pd.read_csv("./prep_table/PREP.feedbacks.csv")
df_learningmaterial = pd.read_csv("./prep_table/PREP.learningmaterials.csv")
df_users = pd.read_csv("./prep_table/PREP.users.csv")

# Define a function to print the schema of a DataFrame
def print_schema(df, name):
    print(f"Schema of {name}:")
    print(df.info())
    print("\n")

# Print the schema for each DataFrame
print_schema(df_selected_scores, "prep_engagements")
print_schema(df_selected_quizzes, "prep_quizzes")
print_schema(df_feedbacks, "prep_feedbacks")
print_schema(df_learningmaterial, "prep_learningmaterials")
print_schema(df_users, "prep_users")

# Rename _id to quizId in the quizzes table to ensure consistency
df_selected_quizzes.rename(columns={'_id': 'quizId'}, inplace=True)

# Create dimension tables
dim_employees = df_users[['employeeId', 'name', 'email', 'role', 'team', 'department']].drop_duplicates()
dim_quizzes = df_selected_quizzes[['quizId', 'title', 'learningMaterial', 'createdAt']].drop_duplicates()
dim_learning_materials = df_learningmaterial[['title', 'createdBy', 'createdAt']].drop_duplicates()

# Create fact tables
fact_engagements = df_selected_scores[['employeeId', 'quizId', 'score', 'timeSpent', 'completed']].copy()
fact_feedback = df_feedbacks[['employeeId', 'learningMaterialTitle', 'rating', 'createdAt']].copy()

# Define the prep_table folder path
prep_table_folder = "./prep_table"


dim_employees.to_csv(os.path.join(prep_table_folder, 'DIM.employees.csv'), index=False)
dim_quizzes.to_csv(os.path.join(prep_table_folder, 'DIM.quizzes.csv'), index=False)
dim_learning_materials.to_csv(os.path.join(prep_table_folder, 'DIM.learning_materials.csv'), index=False)


fact_engagements.to_csv(os.path.join(prep_table_folder, 'FACT.engagements.csv'), index=False)
fact_feedback.to_csv(os.path.join(prep_table_folder, 'FACT.feedback.csv'), index=False)

print("Dimensional and fact tables have been saved successfully.")


Schema of prep_engagements:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   _id         1000 non-null   object
 1   employeeId  1000 non-null   int64 
 2   quizId      1000 non-null   object
 3   score       1000 non-null   int64 
 4   timeSpent   1000 non-null   int64 
 5   completed   1000 non-null   bool  
dtypes: bool(1), int64(3), object(2)
memory usage: 40.2+ KB
None


Schema of prep_quizzes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               1000 non-null   object
 1   title             1000 non-null   object
 2   learningMaterial  1000 non-null   object
 3   createdAt         1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB
None


Schema of prep_feedbacks:
<

**REPORTING LAYER**

**Feedback Summary Report**

In [21]:
fact_feedback

,employeeId,learningMaterialTitle,rating,createdAt
0,106,Despecto necessitatibus aurum summisse labore ...,4,2024-10-07 09:53:23.030000+00:00
1,24,Tener odio cubicularis argentum aperio.,5,2024-10-07 09:53:23.058000+00:00
2,115,Cruentus sol spectaculum.,5,2024-10-07 09:53:23.081000+00:00
3,32,Adipiscor sunt crebro acsi decens quam excepturi.,3,2024-10-07 09:53:23.105000+00:00
4,203,Anser urbanus decet ter ubi vomito decretum vo...,3,2024-10-07 09:53:23.128000+00:00
...,...,...,...,...
995,217,Tego cupiditate demens arcesso supplanto.,1,2024-10-07 09:53:46.759000+00:00
996,17,Curso ultra crinis tripudio celo sui tenetur a...,2,2024-10-07 09:53:46.782000+00:00
997,222,Venio tunc ventosus sunt tabgo ocer demo reici...,2,2024-10-07 09:53:46.806000+00:00
998,182,Asper supplanto auditor corporis deputo civita...,4,2024-10-07 09:53:46.831000+00:00


In [10]:
reporting_folder = "./report_table"

feedback_summary = fact_feedback.groupby('learningMaterialTitle').agg({'rating': 'mean', 'employeeId': 'count'}).reset_index()
feedback_summary.columns = ['learningMaterialTitle', 'average_rating', 'feedback_count']
feedback_summary.to_csv(os.path.join(reporting_folder, 'feedback_summary_report.csv'), index=False)

**User Feedback Summary Report**

In [11]:
# User Feedback Summary
user_feedback_summary = fact_feedback.groupby('employeeId').agg({
    'rating': ['mean', 'count']
}).reset_index()

# Flatten the multi-level columns
user_feedback_summary.columns = ['employeeId', 'average_rating', 'total_feedback_count']

# Merge with employee dimension to get user names
user_feedback_summary = user_feedback_summary.merge(dim_employees[['employeeId', 'name']], on='employeeId', how='left')

# Reorder columns for better readability
user_feedback_summary = user_feedback_summary[['employeeId', 'name', 'total_feedback_count', 'average_rating']]
user_feedback_summary.to_csv(os.path.join(reporting_folder, 'user_feedback_summary_report.csv'), index=False)

**User Engagement Report**

In [12]:
# User Engagement Report
user_engagement_report = fact_engagements.groupby('employeeId').agg({
    'score': ['sum', 'count', 'mean'],  # Total score, total quizzes taken, average score
    'timeSpent': 'sum'                   # Total time spent
}).reset_index()

# Flatten the multi-level columns
user_engagement_report.columns = ['employeeId', 'total_score', 'total_quizzes_taken', 'average_score', 'total_time_spent']

# Merge with employee dimension to get user names and department
user_engagement_report = user_engagement_report.merge(
    dim_employees[['employeeId', 'name', 'department']], 
    on='employeeId', 
    how='left'
)

# Reorder columns for better readability
user_engagement_report = user_engagement_report[['employeeId', 'name', 'department', 'total_quizzes_taken', 'total_score', 'average_score', 'total_time_spent']]


user_engagement_report.to_csv(os.path.join(reporting_folder, 'user_engagement_report.csv'), index=False)

**Creating Engagement Score By Calculating Some Features**

In [13]:
# Read the CSV files
df_user_feedback_summary = pd.read_csv('./report_table/user_feedback_summary_report.csv')
df_engagement_report = pd.read_csv('./report_table/user_engagement_report.csv')

# Merge user_feedback_summary with user_engagement_report on employeeId
merged_df = pd.merge(
    df_user_feedback_summary,
    df_engagement_report,
    on='employeeId',
    suffixes=('_feedback', '_engagement')
)

# Calculate engagement score
merged_df['engagement_score'] = (
    0.2 * merged_df['average_rating'] +
    0.2 * merged_df['total_quizzes_taken'] +
    0.2 * merged_df['average_score'] +
    0.2 * merged_df['total_time_spent'] +
    0.2 * merged_df['total_feedback_count']
)

# Normalize the engagement score and scale by 100
merged_df['engagement_score'] = (
    (merged_df['engagement_score'] - merged_df['engagement_score'].min()) / 
    (merged_df['engagement_score'].max() - merged_df['engagement_score'].min())
) * 100

print(merged_df)


reporting_folder = './report_table'  # Define your reporting folder path
merged_df.to_csv(os.path.join(reporting_folder, 'final_reporting_table.csv'), index=False)

     employeeId         name_feedback  total_feedback_count  average_rating  \
0            11        Mr. Van Howell                    10        1.800000   
1            12  Dr. Emmett Gleichner                     3        2.666667   
2            13  Dr. Alfredo Mosciski                     7        3.714286   
3            14           Lula Harris                     5        3.200000   
4            15     Ricardo Dicki DVM                     5        2.600000   
..          ...                   ...                   ...             ...   
228         246          Essie Mayert                     4        2.500000   
229         247     Marlene Jaskolski                     1        5.000000   
230         248           Bryan Fahey                     2        3.000000   
231         249          May Nitzsche                     3        3.333333   
232         250            Todd Rohan                     1        3.000000   

          name_engagement   department  total_quizz

In [14]:

# Merge data to create the report table
# First, join engagements with employee and quiz details
report_engagements = fact_engagements.merge(dim_employees, on='employeeId', how='left').merge(dim_quizzes, on='quizId', how='left')

# Then, join feedback with employee and learning material details
report_feedback = fact_feedback.merge(dim_employees, on='employeeId', how='left').merge(dim_learning_materials, left_on='learningMaterialTitle', right_on='title', how='left')

# Merge engagements and feedbacks into a final report
report_table = pd.concat([report_engagements, report_feedback], ignore_index=True)


print_schema(report_table, "report_table")




Schema of report_table:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   employeeId             2000 non-null   int64  
 1   quizId                 1000 non-null   object 
 2   score                  1000 non-null   float64
 3   timeSpent              1000 non-null   float64
 4   completed              1000 non-null   object 
 5   name                   2000 non-null   object 
 6   email                  2000 non-null   object 
 7   role                   2000 non-null   object 
 8   team                   2000 non-null   object 
 9   department             2000 non-null   object 
 10  title                  2000 non-null   object 
 11  learningMaterial       1000 non-null   object 
 12  createdAt              1000 non-null   object 
 13  learningMaterialTitle  1000 non-null   object 
 14  rating                 1000 non-

**1. Top 10 employees based on score**

In [15]:
# Load the report_table.csv from the reporting folder
reporting_folder = "./report_table"
report_table = pd.read_csv(os.path.join(reporting_folder, 'reporting_table.csv'))

# Group by employeeId and calculate the mean score
top_10_scores = report_table.groupby('employeeId')['score'].mean().reset_index()

# Sort by score in descending order and get the top 10 employees
top_10_scores = top_10_scores.sort_values(by='score', ascending=False).head(10)

# Merge with report_table to get employee names and other details
top_10_scores = top_10_scores.merge(report_table[['employeeId', 'name']].drop_duplicates(), on='employeeId', how='left')


print("Top 10 Employees Based on Score:")
print(top_10_scores[['name', 'score']])


top_10_scores.to_csv(os.path.join(reporting_folder, 'top_10_employees_by_score.csv'), index=False)


Top 10 Employees Based on Score:
                name      score
0     Dr. Joel Block  97.000000
1  Irving Deckow Jr.  89.333333
2  Mr. Troy Medhurst  88.000000
3  Jacqueline Cronin  84.000000
4       Lucia Wisozk  81.500000
5         Mack Wolff  81.000000
6  Santiago Kassulke  80.000000
7     Stanley Herman  80.000000
8  Roberto Schroeder  79.000000
9        Paul Wehner  78.500000


**2. Top 10 employees based on time spent**

In [16]:

# Load the report_table.csv from the reporting folder
reporting_folder = "./report_table"
report_table = pd.read_csv(os.path.join(reporting_folder, 'reporting_table.csv'))

# Group by employeeId and calculate the mean timeSpent
top_10_time_spent = report_table.groupby('employeeId')['timeSpent'].mean().reset_index()

# Sort by timeSpent in descending order and get the top 10 employees
top_10_time_spent = top_10_time_spent.sort_values(by='timeSpent', ascending=False).head(10)

# Merge with report_table to get employee names and other details
top_10_time_spent = top_10_time_spent.merge(report_table[['employeeId', 'name']].drop_duplicates(), on='employeeId', how='left')


print("\nTop 10 Employees Based on Time Spent:")
print(top_10_time_spent[['name', 'timeSpent']])

top_10_time_spent.to_csv(os.path.join(reporting_folder, 'top_10_employees_by_time_spent.csv'), index=False)



Top 10 Employees Based on Time Spent:
                    name   timeSpent
0         Dr. Joel Block  117.000000
1    Dr. Jennifer Sporer  117.000000
2      Santiago Kassulke  113.000000
3         Fred O'Connell  107.750000
4        Billy Schroeder  106.333333
5         Geraldine West  103.250000
6         Cynthia Russel  102.166667
7  Lynette Streich-Berge  101.000000
8          Wanda Johnson  100.000000
9          Lynette Thiel   99.500000


**3. Top and least title based on feedback**

In [17]:
# Load the report_table.csv from the reporting folder
reporting_folder = "./report_table"
report_table = pd.read_csv(os.path.join(reporting_folder, 'reporting_table.csv'))

# Group feedback by 'learningMaterialTitle' and calculate the average rating for each title
feedback_ratings = report_table.groupby('learningMaterialTitle')['rating'].mean().reset_index(name='average_rating')

# Sort by average rating to find the top and least-rated learning material
top_feedback = feedback_ratings.sort_values(by='average_rating', ascending=False).head(10)
least_feedback = feedback_ratings.sort_values(by='average_rating', ascending=True).head(10)


print("\nTop Learning Material Title by Average Rating:")
print(top_feedback)

print("\nLeast Learning Material Title by Average Rating:")
print(least_feedback)


top_feedback.to_csv(os.path.join(reporting_folder, 'top_10_Courses_by_feedback.csv'), index=False)
least_feedback.to_csv(os.path.join(reporting_folder, 'least_10_Courses_by_feedback.csv'), index=False)


Top Learning Material Title by Average Rating:
                                 learningMaterialTitle  average_rating
376  Quia adulescens validus deporto circumvenio ac...             5.0
619  Vorago unde voveo tribuo creta complectus tondeo.             5.0
372   Praesentium auctus addo solus adduco non stipes.             5.0
625                            Vulgo subiungo tamquam.             5.0
591  Vilitas cena tabesco benigne vobis tenus subit...             5.0
582   Vetus auxilium absum tardus qui concedo allatus.             5.0
357                        Pariatur temptatio dapifer.             5.0
390  Reprehenderit testimonium synagoga clibanus de...             5.0
19                                 Adhaero sto aufero.             5.0
378  Quis demoror aut pecus natus sursum eveniet st...             5.0

Least Learning Material Title by Average Rating:
                                 learningMaterialTitle  average_rating
627  Vulnus tandem assumenda solvo delego dolor ac

**4. Titles of learning material with highest and lowest average score**

In [18]:
# Group scores by 'quizId' and calculate the average score for each quiz
average_scores = report_table.groupby('quizId')['score'].mean().reset_index()

# Merge with report_table to get the titles associated with the quizzes
average_scores = average_scores.merge(report_table[['title', 'quizId']], left_on='quizId', right_on='quizId', how='left')

# Sort by average score in descending order
average_scores_sorted = average_scores.sort_values(by='score', ascending=False)

# Get top 10 titles with the highest average scores
top_10_titles = average_scores_sorted.head(10)

# Get last 10 titles with the lowest average scores
last_10_titles = average_scores_sorted.tail(10)


print("\nTop 10 Learning Material Titles by Average Score:")
print(top_10_titles[['title', 'score']])

print("\nLast 10 Learning Material Titles by Average Score:")
print(last_10_titles[['title', 'score']])


top_10_titles[['title', 'score']].to_csv(os.path.join(reporting_folder, 'Top_10_Courses_based_on_score.csv'), index=False)
last_10_titles[['title', 'score']].to_csv(os.path.join(reporting_folder, 'Last_10_Courses_based_on_score.csv'), index=False)


Top 10 Learning Material Titles by Average Score:
                                                 title  score
928                             Valeo arguo temeritas.  100.0
861  Curatio sed solio utique voluptate adsidue the...  100.0
680                        Alo commodo placeat vitium.  100.0
61   Copiose utilis tricesimus turbo comis certus p...  100.0
859        Curiositas vita aeneus cunabula ut ascisco.  100.0
358  Vere aufero ad subseco alii aspernatur curto b...   99.0
69   Utor similique una suppono adaugeo consuasor n...   99.0
302                      Titulus tremo depraedor quod.   98.0
745  Valde tum appositus clam admoveo rem depraedor...   98.0
941  Uter tener libero arbustum temeritas ullam adf...   98.0

Last 10 Learning Material Titles by Average Score:
                                                 title  score
671  Delectus pariatur basium earum velociter ambit...    1.0
33   Templum inventore eum subvenio aranea clibanus...    1.0
406                          

**5. Department performance based on average score**

In [19]:
# Merge employee information with scores based on employeeId
department_performance = report_table[['employeeId', 'department']].merge(report_table[['employeeId', 'score']], on='employeeId', how='left')

# Calculate the average score for each department
average_department_scores = department_performance.groupby('department')['score'].mean().reset_index()

# Sort the departments by average score
average_department_scores_sorted = average_department_scores.sort_values(by='score', ascending=False)

# Get top 10 best performing departments
top_10_departments = average_department_scores_sorted.head(10)

# Get bottom 10 least performing departments
bottom_10_departments = average_department_scores_sorted.tail(10)


print("\nTop 10 Best Performing Departments by Average Score:")
print(top_10_departments)

print("\nBottom 10 Least Performing Departments by Average Score:")
print(bottom_10_departments)


top_10_departments[['department', 'score']].to_csv(os.path.join(reporting_folder, 'best_performing_department.csv'), index=False)
bottom_10_departments[['department', 'score']].to_csv(os.path.join(reporting_folder, 'worst_performing_department.csv'), index=False)


Top 10 Best Performing Departments by Average Score:
    department      score
5    Computers  62.265176
10      Health  56.209402
3        Books  52.509554
21        Toys  51.752556
11        Home  50.580952
14        Kids  49.460976
16       Music  48.604430
18       Shoes  48.010601
4     Clothing  47.975976
12  Industrial  47.712171

Bottom 10 Least Performing Departments by Average Score:
    department      score
0   Automotive  46.970149
8       Garden  46.761905
9      Grocery  46.016835
1         Baby  45.937669
15      Movies  45.584746
20       Tools  45.565909
13     Jewelry  43.505464
19      Sports  42.831120
17    Outdoors  40.085791
7        Games  31.631399
